In [1]:
using CSV, DataFrames, Dates, HiGHS, JuMP, Statistics, Plots

In [2]:
df = CSV.read("data.csv", DataFrame)
D_t = df[!, "FR_load_forecast_entsoe_transparency"]

println(size(D_t))
println(first(D_t, 5))

df = CSV.read("data.csv", DataFrame)
D_t = df[!, "FR_load_forecast_entsoe_transparency"]

println(size(D_t))
println(first(D_t, 5))
        
file_path = "capacity_pv.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_pv = Array(df)
println(first(capacity_pv, 10))
    
file_path = "capacity_wind.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_wind = Array(df)
println(first(capacity_wind, 10))

(8760,)
[56250.0, 54300.0, 53600.0, 50000.0, 47100.0]
(8760,)
[56250.0, 54300.0, 53600.0, 50000.0, 47100.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.067, 0.218]
[0.12594, 0.123693, 0.124232, 0.13045, 0.140045, 0.136766, 0.129185, 0.135791, 0.152821, 0.14136]


In [3]:
file_path = "Imbalance.csv"
data = CSV.read(file_path, DataFrame)

# The column containing the target data
imbalance_column_name = "Total Imbalance [MWh] - SCA|FR"

# Check if the column is present
if !(imbalance_column_name in names(data))
    error("Column '$imbalance_column_name' not found in the data.")
end

# Extract the "Total Imbalance [MWh] - SCA|FR" data
imbalance_data = data[!, imbalance_column_name]

# Convert non-numeric values to missing values
function to_int(value)
    try
        return parse(Int, value)
    catch
        return missing
    end
end

# Apply the conversion function and filter out missing values
imbalance = filter(!ismissing, to_int.(imbalance_data))

# imbalance = first(imbalance,8760*2)


17212-element Vector{Union{Missing, Int64}}:
 489
 922
 630
 416
 356
 547
  68
 200
 408
 362
   ⋮
 486
 226
 363
 128
 241
 288
  47
 702
 506

In [5]:
# # Step 6: Compute the integral under the ORDC curve using the rectangles method
# discretization_step = 20
# # ordc_utility = Float64[]
# U_ORDC= Float64[]
# for R in reserve_range
#     # Define the interval for the current reserve R
#     intervals = collect(0:discretization_step:R)
    
#     # Calculate the area under the curve for the current reserve R using rectangles method
#     area = 0.0
#     for i in 1:length(intervals)-1
#         height = ordc_values[findfirst(x -> x ≥ intervals[i], reserve_range)]
#         width = intervals[i+1] - intervals[i]
#         area += height * width
#     end
#     # push!(ordc_utility, area)
#     push!(U_ORDC, area)
# end


In [6]:
# Step 6: Compute the integral under the ORDC curve using the rectangles method
# Sort the data to calculate the cumulative probability
# sorted_data = sort(imbalance)
sorted_data = sort(vec(sum(reshape(imbalance, 2, :), dims=1)))

function compute_lolp(reserve, sorted_imbalance)
    # Probability of an imbalance higher than the given reserve
    return mean(sorted_imbalance .> reserve)
end


discretization_step = 20
# ordc_utility = Float64[]
R_min_shift = minimum(sorted_data)
R_max_shift = maximum(sorted_data)
reserve_range = R_min_shift:discretization_step:R_max_shift


lolp_values = [compute_lolp(R, sorted_data) for R in reserve_range]

VOLL = 5000

ordc_values = lolp_values .* VOLL

U_ORDC= Float64[]
for R in reserve_range
    # Define the interval for the current reserve R
    intervals = collect(0:discretization_step:R)
    
    # Calculate the area under the curve for the current reserve R using rectangles method
    area = 0.0
    for i in 1:length(intervals)-1
        height = ordc_values[findfirst(x -> x ≥ intervals[i], reserve_range)]
        width = intervals[i+1] - intervals[i]
        area += height * width
    end
    # push!(ordc_utility, area)
    push!(U_ORDC, area)
end

R_values = [R - R_min_shift for R in reserve_range]
R_min = R_values[1]
R_max = R_values[end]
n_R_val = length(R_values)

260

In [7]:
# Constants and Data
T = 8760
lambda = 5000  
cost_of_debt = 0.04
cost_of_equity = 0.07
corporate_tax = 0.30
economic_life = 20
carbon_tax = 50 

capacity = Dict(
    "Coal" => 1,
    "CCGT" => 1,
    "OCGT" => 1,
    "Onshore Wind" => capacity_wind,
    "Offshore Wind" => capacity_wind,
    "PV" => capacity_pv
)

# Technologies data
technologies = Dict(
    "Coal" => (capex=2000*1000, om=0.03, debt_ratio=0.62, heat_rate=2.4, EA=175.2252157*1000, price =5, C=12, emissions=1.4),
    "CCGT" => (capex=950*1000, om=0.03, debt_ratio=0.56, heat_rate=1.62, EA=85.60445144*1000, price =30, C=48.6, emissions=0.5),
    "OCGT" => (capex=700*1000, om=0.03, debt_ratio=0.6, heat_rate=2.5, EA=61.91153839*1000, price =30, C=75, emissions=0.6),
    "Onshore Wind" => (capex=700*1000, om=0.03, debt_ratio=0.7, heat_rate=0, EA=58.99797384*1000, price =0, C=0, emissions=0.0),
    "Offshore Wind" => (capex=1300*1000, om=0.03, debt_ratio=0.7, heat_rate=0, EA=109.5676657*1000, price =0, C=0, emissions=0.0),
    "PV" => (capex=400*1000, om=0.03, debt_ratio=0.8, heat_rate=0, EA=32.04823387*1000, price =0, C=0, emissions=0.0)
)

technologies_t = Dict(
    "Coal" => (capex=2000*1000, om=0.03, debt_ratio=0.62, heat_rate=2.4, EA=175.2252157*1000, price =5, C=12, emissions=1.4),
    "CCGT" => (capex=950*1000, om=0.03, debt_ratio=0.56, heat_rate=1.62, EA=85.60445144*1000, price =30, C=48.6, emissions=0.5),
    "OCGT" => (capex=700*1000, om=0.03, debt_ratio=0.6, heat_rate=2.5, EA=61.91153839*1000, price =30, C=75, emissions=0.6)
)

#Battery data
F_b=6*1000/20
A_b=1.620627574*1000
eta=0.9

0.9

In [8]:
T = 8760
model = Model(HiGHS.Optimizer)


@variable(model, k[g in keys(technologies)] >= 0)  # Installed capacity
@variable(model, p[g in keys(technologies), t=1:T] >= 0)  # Hourly production
@variable(model, r[g in keys(technologies_t), t=1:T] >= 0)  # Hourly reserve # seulement pour les thermiques
# @variable(model, R[t=1:T]>=0) #Total Hourly Reserve
@variable(model, z[t=1:T, i=1:n_R_val], Bin)  # Binary variable for reserve selection
@variable(model, d[t=1:T] >=0)
#For Battery
@variable(model, k_b>=0 ) 
@variable(model, e[1:T] >= 0)
@variable(model, p_in[1:T] >=0)
@variable(model, p_out[1:T] >=0)



# Define the objective function
@objective(model, Min,
    sum((technologies[g].om * technologies[g].capex + technologies[g].EA) * k[g] +
        sum(technologies[g].C * p[g, t] for t in 1:T) for g in keys(technologies)) +
    (A_b + F_b) * k_b -
    sum(lambda * d[t] for t in 1:T) +
    carbon_tax * sum(technologies[g].emissions * sum(p[g, t] for t in 1:T) for g in keys(technologies)) -
    sum(U_ORDC[i] * z[t, i] for t in 1:T, i = 1:n_R_val)
)
#- sum(U_ORDC[R[t]] for t=1:T))
# + sum(LOLP[t] * VOLL * r[g, t] for g in keys(technologies_t), t=1:T) 
    
@constraint(model, [ g in keys(technologies_t), t in 1:T], p[g,t] <= k[g] - r[g,t])
@constraint(model, [ g in keys(technologies), t in 1:T], p[g,t] <= k[g])
@constraint(model, [t in 1:T], d[t]<=D_t[t])
@constraint(model, [t in 1:T], -d[t] + p_out[t] - p_in[t]  + sum( p[g,t] for g in keys(technologies))==0)
@constraint(model, [t in 1:T], e[t]<= 4*k_b)
@constraint(model, [t in 1:T], p_in[t]<=k_b)
@constraint(model, [t in 1:T], p_out[t]<=k_b)
@constraint(model,  e[1]==0)
@constraint(model, [t in 2:T], e[t]== e[t-1] + 1*(sqrt(eta)* p_in[t-1] - 1/sqrt(eta) * p_out[t-1]))
for g in ["Coal", "CCGT", "OCGT"]
    @constraint(model, [t in 1:T], p[g, t] + r[g, t] <= k[g])
end
for t in 1:T
    @constraint(model, p["Onshore Wind", t] <= capacity_wind[t]*k["Onshore Wind"])
    @constraint(model, p["Offshore Wind", t] <= capacity_wind[t]*k["Offshore Wind"])
    @constraint(model, p["PV", t] <= capacity_pv[t]*k["PV"])
end

#reserve constraints
@constraint(model, [t in 1:T], sum(r[g,t] for g in keys(technologies_t))==sum(R_values[i] * z[t, i] for i in 1:n_R_val))
# Ensure exactly one `z[t, i]` is 1 for each `t`
@constraint(model, [t = 1:T], sum(z[t, i] for i = 1:n_R_val) == 1)



optimize!(model)

print("Variables value, k:",value.(k))
print("Variables value, k_b:",value.(k_b))
# print("Variables value, d:",value.(d))
print("Objective value: ", objective_value(model))
print(keys(technologies))